# Accelerations and decelerations from Taylor et al. 2000

[Taylor et al. 2000](https://doi.org/10.1111/j.1471-0528.2000.tb11112.x) - The development and validation of an algorithm for real-time computerised fetal heart rate monitoring in labour

**Cleaning signals:** "Both the fetal heart rate and uterine pressure signals contained artefact... Artefact must be removed prior to determining the timing of the decelerations; this is achieved by a low pass fourth order Butterworth filter with a cut off frequency of 0.02 Hz"

**Identifying accelerations and decelerations:** "Accelerations and decelerations were determined and classified according to definitions given by FIGO."

Not performed in FHRMA but also in paper:
* **Classifying decelerations:** "The classification of decelerations was based upon the temporal relationship between the minimum of the deceleration and the maximum of the associated uterine contraction. The uterine contractions were timed by means of an external tocograph. A late deceleration occurred where its minimum value was 20–60 s after the peak of a contraction"
* **Calculating FHR variability:** "The fetal heart rate variability is calculated as twice the standard deviation of the fetal heart rate recorded continuously, after removing the accelerations and decelerations. Reduced variability is defined as one standard deviation < 5 beats per minute."

As you can see, the description of the method for identifying accelerations and decelerations is not clear, and I have followed the interpretation of Boudet et al. below.

### Implementation in MATLAB

Boudet et al. implement Taylor et al. in the FHRMA toolbox, as in [this MATLAB function](https://github.com/utsb-fmm/FHRMA/blob/master/aamtaylor.m) which is copied below.

```
FuzzyLine=butterfilt(FHRi,4,0,0.02,4,1);

% The following is not clearly described on the paper. 
% The begining and ending of accident is described for the baseline caluclation but
% not directly for the accident determination which we guess should be
% different. Here is the sentence which let us suppose our code.
% 
% "Accelerations and decelerations were determined and classified according
% to definitions given by FIGO" 
% "Artefact lust be removed prior to determining the timing of the
% decelerations; this is achieved by a low pass fourth order Butterworth
% filter with a cut off frequency of 0.02Hz"
% Context on those sentences have ambiguities so there is some doubts

if( exist('simpleacc','var') && simpleacc==1)
    [accelerations,decelerations]=simpleaddetection(FHR,baseline);

else
    %List of periods where there is 10bpm difference which are acceleration
    %candidats
    accelerations=startendlist(FuzzyLine-baseline>10)/4; 
    %Delete periods of too short duration or too low amplitude
    accelerations=validaccident(accelerations,FHRi-baseline,15,15); 


    decelerations=startendlist(FuzzyLine-baseline<-5)/4;
    decelerations=validaccident(decelerations,baseline-FHRi,15,15);
end
``````

Their function for simple accleration and deceleration detection:

```
% TRivial method for Acceleration/Deceleration detection
%
% USAGE
%    [acc,dec,falseacc,falsedec]=simpleaddetection(fhr,baseline)
%         
%
% INPUT
%     FHR       : Fetal Heart Rate sampled at 4Hz
%     baseline  : baseline sampled at 4Hz
%
% OUTPUT
%     acc       : 3xn for each n acceleration [start end max]
%     dec       : 3xn for each n deceleraiton [start end max]
%     falseacc  : Same for acceleration with only 5 bpm high
%     falsedec  : Same for deceleration with only 5 bpm high
% 

%123456789012345678901234567890123456789012345678901234567890123456789012
%
%     FHR Morphological Analysis Toolbox  Copyright (C) 2018 Samuel Boudet, Faculté de Médecine et Maïeutique,
%     samuel.boudet@gmail.com
%
%     This file is part of FHR Morphological Analysis Toolbox 
%
%     FHR Morphological Analysis Toolbox  is free software: you can redistribute it and/or modify
%     it under the terms of the GNU General Public License as published by
%     the Free Software Foundation, either version 3 of the License, or
%     (at your option) any later version.
% 
%     FHR Morphological Analysis Toolbox  is distributed in the hope that it will be useful,
%     but WITHOUT ANY WARRANTY; without even the implied warranty of
%     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
%     GNU General Public License for more details.
% 
%     You should have received a copy of the GNU General Public License
%     along with this program.  If not, see <http://www.gnu.org/licenses/>.
%



function [acc,dec,falseacc,falsedec]=simpleaddetection(fhr,baseline)

acc=detectaccident(fhr-baseline,15);
dec=detectaccident(baseline-fhr,15);
falseacc=minusint(acc,detectaccident(fhr-baseline,5));
falsedec=minusint(dec,detectaccident(baseline-fhr,5));
end

function accidentsample=detectaccident(sig,thre)

peaks=find(sig>thre);
accidentsample=zeros(3,0);
while ~isempty(peaks)
    dacc=find(sig(1:peaks)<0,1,'last');
    if isempty(dacc)
        dacc=1;
    end
    facc=find(sig(dacc+1:end)<0,1,'first')+dacc;
    if isempty(facc)
        facc=length(sig);
    end
    [~,macc]=max(sig(dacc:facc));
    macc=macc+dacc-1;
    if facc-dacc>15*4
        accidentsample=[accidentsample [dacc;facc;macc]/4];
    end   
    peaks=peaks(peaks>facc);
end
end

function f=minusint(a,f)

for i=1:size(a,2)
    n=find(f(1,:)>=a(1,i) &f(2,:)<=a(2,i));
    if ~isempty(n)
        f=f(:,[1:n-1 n+1:end]);
    end
end
    
end
```

In [1]:
# Not yet implemented